In [1]:
import simcat   
import toytree    ## requires github Master branch currently
import random
import numpy as np
from copy import deepcopy
import toyplot

In [2]:
def sample_branches(toytreeobj, betaval = 1,returndict = None):
    """
    Takes an input topology and samples branch lengths
    
    Parameters:
    -----------
    toytreeobj: toytree
        The topology for which we want to generate branch lengths
    betaval: int/float (default=1)
        The value of beta for the exponential distribution used to generate
        branch lengths. This is inverse of rate parameter.
    returndict: str (default=None)
        If "only", returns only a dictionary matching nodes to heights.
        If "both", returns toytree and dictionary as a tuple.
    """
    tree=deepcopy(toytreeobj)
    def set_node_height(node,height):
        childn=node.get_children()
        for child in childn:
            child.dist=height - child.height
    testobj = []
    
    # we'll just append each step on each chain to these lists to keep track of where we've already been
    allnodes = []
    allnodeheights = []
    
    # d is deprecated (it was being returned) and stores the values for each chain
    #d = {}
    
    ## testobj is our traversed full tree
    for i in tree.tree.traverse():
        testobj.append(i)
    
    # start by getting branch lengths for the longest branch
    longbranch=sum([testobj[-1] in i for i in testobj]) # counts the number of subtrees containing last leaf, giving length of longest branch
    longbranchnodes = np.random.exponential(betaval,longbranch) # segment lengths of longest branch
    longbranchnodes[0] = np.random.uniform(low=0.0,high=longbranchnodes[0]) # cut the edge to the leaf with a uniform draw
    
    # get the heights of nodes along this longest (most nodes) branch
    nodeheights = []
    currheight = 0
    for i in longbranchnodes:
        currheight += i
        nodeheights.append(currheight)
    nodeheights = nodeheights[::-1]

    # get indices to accompany long chain
    lcidx = []
    for i in testobj:
        if testobj[-1] in i.get_leaves():
            lcidx.append(i.idx)
    #d['0heights'] = np.array(nodeheights)
    #d['0nodes'] = np.array(lcidx[:-1])
    
    allnodes = allnodes + lcidx[:-1]
    allnodeheights = allnodeheights + nodeheights
    
    # get other necessary chains to parse
    other_chains = []
    for i in range(len(testobj)-1)[::2]:
        if len(testobj[i+1].get_leaves()) > 1:
            other_chains.append(testobj[i+1])

    # now solve
    for chainnum in range(len(other_chains)): # parse the remaining chains one at a time
        otr=other_chains[chainnum]
        # find where this chain connects to the a chain we've already solved
        firstancestor = otr.get_ancestors()[0].idx
        # which nodeheight does this branch from
        paridx=np.argmax(np.array(allnodes) == firstancestor) 
        
        # traverse the new 
        testobj1 = []
        nodes = []
        
        # save a list of nodes
        for i in otr.traverse():
            testobj1.append(i)
        
        # save the nodes that include the end of the chain (because branches out to other chains might not)
        for i in testobj1:
            if testobj1[-1] in i.get_leaves():
                nodes.append(i.idx)
        
        # make node index list to accompany lengths
        lennodes= nodes[:-1] # don't save ending leaf index
        lennodes.insert(0,firstancestor) # make chain list start with ancestor
        
        # figure out how many exponential draws to make for this chain (i.e. # new nodes + 1)
        num_new_branches = sum([testobj1[-1] in i for i in testobj1])+1
        
        # initialize array to hold the draws
        mir_lens = np.zeros((sum([testobj1[-1] in i for i in testobj1])+1))
        # draw until we have a new set of exponential branch lengths that fit the constraints of our tree height
        while not (sum(mir_lens[:(len(mir_lens)-1)]) < allnodeheights[paridx] and (sum(mir_lens) > allnodeheights[paridx])):
            mir_lens = np.random.exponential(betaval,num_new_branches) ## length of longest branch
        
        # now let's save each node value as a height
        mir_lens_heights = np.zeros((len(mir_lens)))
        subsum = 0
        for i in range(len(mir_lens)):
            mir_lens_heights[i] = allnodeheights[paridx] - subsum
            subsum = subsum + mir_lens[i]
            
        # add our new node indices with their heights to the full list
        allnodes = list(allnodes) + list(lennodes)
        allnodeheights = list(allnodeheights) + list(mir_lens_heights)
        
        #d[(str(chainnum+1)+"heights")] = mir_lens_heights
        #d[(str(chainnum+1)+"nodes")] = np.array(lennodes)
    
    # make a final dictionary of node heights, eliminating redundancy
    n = dict(set(zip(*[allnodes,allnodeheights])))
    
    if returndict == "only":
        return n #d
    elif returndict == "both":
        # create the tree object
        for node in n.keys():
            set_node_height(tree.tree.search_nodes(idx = node)[0],n[node])
        return (tree,n)
    else:
        # create the tree object
        for node in n.keys():
            set_node_height(tree.tree.search_nodes(idx = node)[0],n[node])
        return tree

In [3]:
## generate a random tree
random.seed(123)
tree = toytree.rtree(10)
tree.tree.convert_to_ultrametric()
tree.draw(tree_style='c', node_labels='idx', tip_labels=False, padding=50);

idx: 0
name: t-9
dist: 1
support: 100
height: 0 0 idx: 1
name: t-8
dist: 1
support: 100
height: 0 1 idx: 2
name: t-7
dist: 2
support: 100
height: 0 2 idx: 3
name: t-6
dist: 1
support: 100
height: 0 3 idx: 4
name: t-5
dist: 1
support: 100
height: 0 4 idx: 5
name: t-4
dist: 2
support: 100
height: 0 5 idx: 6
name: t-3
dist: 1
support: 100
height: 0 6 idx: 7
name: t-2
dist: 1
support: 100
height: 0 7 idx: 8
name: t-1
dist: 2
support: 100
height: 0 8 idx: 9
name: t-0
dist: 3
support: 100
height: 0 9 idx: 10
name: i10
dist: 1
support: 1
height: 1 10 idx: 13
name: i13
dist: 1
support: 1
height: 2 14 idx: 11
name: i11
dist: 1
support: 1
height: 1 13 idx: 14
name: i14
dist: 1
support: 1
height: 2 16 idx: 16
name: i16
dist: 1
support: 1
height: 3 15 idx: 12
name: i12
dist: 1
support: 1
height: 1 11 idx: 15
name: i15
dist: 1
support: 1
height: 2 12 idx: 17
name: i17
dist: 1
support: 1
height: 3 17 idx: 18
name: i18
dist: 0
support: 100
height: 4 18

### We can now sample new branch lengths for this topology, returning a toytree object:

In [4]:
sample_branches(tree)

### Or a dictionary of node heights:

In [5]:
sample_branches(tree,returndict="only")

{10: 0.094012886195441472,
 11: 0.28192250912238759,
 12: 3.8897344254572359,
 13: 1.0257088860992656,
 14: 3.3146854292794794,
 15: 5.0212391885370202,
 16: 4.9268011066444197,
 17: 5.0592015926470788,
 18: 5.2988060062717395}

### Do this several times:

In [6]:
ntrees = 4
canvas = toyplot.Canvas(width=800, height=200)
axes = [canvas.cartesian(grid=(1, ntrees, idx)) for idx in range(ntrees)]
for i in range(ntrees):
    ax = axes[i]
    ax.show = False
    sample_branches(tree).draw(
        axes=ax, tree_style='c', node_labels='name', tip_labels=False, node_size=16)

idx: 0
name: t-9
dist: 0.550000803661
support: 100
height: 0.0 t-9 idx: 1
name: t-8
dist: 0.550000803661
support: 100
height: 0.0 t-8 idx: 2
name: t-7
dist: 1.65230412162
support: 100
height: 0.0 t-7 idx: 3
name: t-6
dist: 2.03775629956
support: 100
height: 0.0 t-6 idx: 4
name: t-5
dist: 2.03775629956
support: 100
height: 0.0 t-5 idx: 5
name: t-4
dist: 2.77428729273
support: 100
height: 0.0 t-4 idx: 6
name: t-3
dist: 0.484719614129
support: 100
height: 0.0 t-3 idx: 7
name: t-2
dist: 0.484719614129
support: 100
height: 0.0 t-2 idx: 8
name: t-1
dist: 1.59243478263
support: 100
height: 0.0 t-1 idx: 9
name: t-0
dist: 1.78247031608
support: 100
height: 0.0 t-0 idx: 10
name: i10
dist: 1.10230331796
support: 1
height: 0.550000803661 i10 idx: 13
name: i13
dist: 1.70472993743
support: 1
height: 1.65230412162 i14 idx: 11
name: i11
dist: 0.736530993163
support: 1
height: 2.03775629956 i13 idx: 14
name: i14
dist: 0.582746766327
support: 1
height: 2.77428729273 i16 idx: 16
name: i16
dist: 1.16308466497
support: 1
height: 3.35703405905 i15 idx: 12
name: i12
dist: 1.1077151685
support: 1
height: 0.484719614129 i11 idx: 15
name: i15
dist: 0.19003553345
support: 1
height: 1.59243478263 i12 idx: 17
name: i17
dist: 2.73764840794
support: 1
height: 1.78247031608 i17 idx: 18
name: i18
dist: 0.0
support: 100
height: 4.52011872402 i18 idx: 0
name: t-9
dist: 1.22452190603
support: 100
height: 0.0 t-9 idx: 1
name: t-8
dist: 1.22452190603
support: 100
height: 0.0 t-8 idx: 2
name: t-7
dist: 1.69610734765
support: 100
height: 0.0 t-7 idx: 3
name: t-6
dist: 2.20324793117
support: 100
height: 0.0 t-6 idx: 4
name: t-5
dist: 2.20324793117
support: 100
height: 0.0 t-5 idx: 5
name: t-4
dist: 2.3506949772
support: 100
height: 0.0 t-4 idx: 6
name: t-3
dist: 0.00726218361386
support: 100
height: 0.0 t-3 idx: 7
name: t-2
dist: 0.00726218361386
support: 100
height: 0.0 t-2 idx: 8
name: t-1
dist: 0.30922502632
support: 100
height: 0.0 t-1 idx: 9
name: t-0
dist: 0.621423986065
support: 100
height: 0.0 t-0 idx: 10
name: i10
dist: 0.471585441626
support: 1
height: 1.22452190603 i10 idx: 13
name: i13
dist: 1.07226131003
support: 1
height: 1.69610734765 i14 idx: 11
name: i11
dist: 0.147447046022
support: 1
height: 2.20324793117 i13 idx: 14
name: i14
dist: 0.417673680493
support: 1
height: 2.3506949772 i16 idx: 16
name: i16
dist: 0.0845156215578
support: 1
height: 2.76836865769 i15 idx: 12
name: i12
dist: 0.301962842706
support: 1
height: 0.00726218361386 i11 idx: 15
name: i15
dist: 0.312198959746
support: 1
height: 0.30922502632 i12 idx: 17
name: i17
dist: 2.23146029318
support: 1
height: 0.621423986065 i17 idx: 18
name: i18
dist: 0.0
support: 100
height: 2.85288427925 i18 idx: 0
name: t-9
dist: 0.560250496682
support: 100
height: 0.0 t-9 idx: 1
name: t-8
dist: 0.560250496682
support: 100
height: 0.0 t-8 idx: 2
name: t-7
dist: 1.15150893223
support: 100
height: 0.0 t-7 idx: 3
name: t-6
dist: 0.190747799084
support: 100
height: 0.0 t-6 idx: 4
name: t-5
dist: 0.190747799084
support: 100
height: 0.0 t-5 idx: 5
name: t-4
dist: 0.760462401554
support: 100
height: 0.0 t-4 idx: 6
name: t-3
dist: 0.322055957527
support: 100
height: 0.0 t-3 idx: 7
name: t-2
dist: 0.322055957527
support: 100
height: 0.0 t-2 idx: 8
name: t-1
dist: 0.367221681296
support: 100
height: 0.0 t-1 idx: 9
name: t-0
dist: 2.43333476314
support: 100
height: 0.0 t-0 idx: 10
name: i10
dist: 0.591258435545
support: 1
height: 0.560250496682 i10 idx: 13
name: i13
dist: 0.601395887101
support: 1
height: 1.15150893223 i14 idx: 11
name: i11
dist: 0.56971460247
support: 1
height: 0.190747799084 i13 idx: 14
name: i14
dist: 0.992442417774
support: 1
height: 0.760462401554 i16 idx: 16
name: i16
dist: 0.863823981858
support: 1
height: 1.75290481933 i15 idx: 12
name: i12
dist: 0.0451657237688
support: 1
height: 0.322055957527 i11 idx: 15
name: i15
dist: 2.06611308185
support: 1
height: 0.367221681296 i12 idx: 17
name: i17
dist: 0.183394038042
support: 1
height: 2.43333476314 i17 idx: 18
name: i18
dist: 0.0
support: 100
he

### Currently, the toytree object part breaks when the number of taxa gets above 16 -- the tips no longer line up.

In [7]:
random.seed(123)
tree = toytree.rtree(16)
tree.tree.convert_to_ultrametric()
sample_branches(tree).draw(tree_style='c', node_labels='idx', tip_labels=False, padding=50);

idx: 0
name: t-15
dist: 1.66440267045
support: 100
height: 0.0 0 idx: 1
name: t-14
dist: 1.66440267045
support: 100
height: 0.0 1 idx: 2
name: t-13
dist: 1.3421138283
support: 100
height: 0.0 2 idx: 3
name: t-12
dist: 1.3421138283
support: 100
height: 0.0 3 idx: 4
name: t-11
dist: 0.161233939813
support: 100
height: 0.0 4 idx: 5
name: t-10
dist: 0.161233939813
support: 100
height: 0.0 5 idx: 6
name: t-9
dist: 0.732848048062
support: 100
height: 0.0 6 idx: 7
name: t-8
dist: 0.732848048062
support: 100
height: 0.0 7 idx: 8
name: t-7
dist: 2.30873038423
support: 100
height: 0.0 8 idx: 9
name: t-6
dist: 2.22224936512
support: 100
height: 0.0 9 idx: 10
name: t-5
dist: 2.22224936512
support: 100
height: 0.0 10 idx: 11
name: t-4
dist: 0.112447923901
support: 100
height: 0.0 11 idx: 12
name: t-3
dist: 0.112447923901
support: 100
height: 0.0 12 idx: 13
name: t-2
dist: 2.49262984809
support: 100
height: 0.0 13 idx: 14
name: t-1
dist: 2.99745479506
support: 100
height: 0.0 14 idx: 15
name: t-0
dist: 2.99745479506
support: 100
height: 0.0 15 idx: 16
name: i16
dist: 0.289036141769
support: 1
height: 1.66440267045 16 idx: 17
name: i17
dist: 0.611324983914
support: 1
height: 1.3421138283 17 idx: 19
name: i19
dist: 0.737844470835
support: 1
height: 1.95343881222 20 idx: 20
name: i20
dist: 2.53004934324
support: 1
height: 0.161233939813 24 idx: 24
name: i24
dist: 0.133453005922
support: 1
height: 2.69128328305 25 idx: 18
name: i18
dist: 1.57588233617
support: 1
height: 0.732848048062 19 idx: 21
name: i21
dist: 0.000615129231079
support: 1
height: 2.30873038423 18 idx: 22
name: i22
dist: 0.0870961483417
support: 1
height: 2.22224936512 21 idx: 25
name: i25
dist: 0.515390775512
support: 1
height: 2.30934551346 28 idx: 28
name: i28
dist: 2.65055508783
support: 1
height: 2.82473628897 27 idx: 23
name: i23
dist: 2.38018192419
support: 1
height: 0.112447923901 22 idx: 26
name: i26
dist: 1.71480789287
support: 1
height: 2.49262984809 23 idx: 27
name: i27
dist: 1.2099829459
support: 1
height: 2.99745479506 26 idx: 29
name: i29
dist: 1.26785363584
support: 1
height: 4.20743774096 29 idx: 30
name: i30
dist: 0.0
support: 100
height: 5.4752913768 30

In [8]:
random.seed(123)
tree = toytree.rtree(17)
tree.tree.convert_to_ultrametric()
sample_branches(tree).draw(tree_style='c', node_labels='idx', tip_labels=False, padding=50);

idx: 0
name: t-16
dist: 0.457249308377
support: 100
height: 0.0 0 idx: 1
name: t-15
dist: 0.457249308377
support: 100
height: 0.0 1 idx: 2
name: t-14
dist: 0.655569615686
support: 100
height: 0.0 2 idx: 3
name: t-13
dist: 0.655569615686
support: 100
height: 0.0 3 idx: 4
name: t-12
dist: 1.4262734111
support: 100
height: 0.0 4 idx: 5
name: t-11
dist: 1.4262734111
support: 100
height: 0.0 5 idx: 6
name: t-10
dist: 0.432209915217
support: 100
height: 0.0 6 idx: 7
name: t-9
dist: 0.432209915217
support: 100
height: 0.0 7 idx: 8
name: t-8
dist: 0.631791868242
support: 100
height: 0.0 8 idx: 9
name: t-7
dist: 0.631791868242
support: 100
height: 0.0 9 idx: 10
name: t-6
dist: 0.833990269474
support: 100
height: 0.0 10 idx: 11
name: t-5
dist: 0.833990269474
support: 100
height: 0.0 11 idx: 12
name: t-4
dist: 0.249037539719
support: 100
height: 0.0 12 idx: 13
name: t-3
dist: 0.249037539719
support: 100
height: 0.0 13 idx: 14
name: t-2
dist: 3.17337137686
support: 100
height: 0.0 14 idx: 15
name: t-1
dist: 1.44542372409
support: 100
height: 0.0 15 idx: 16
name: t-0
dist: 1.44542372409
support: 100
height: 0.0 16 idx: 17
name: i17
dist: 0.425707667789
support: 1
height: 0.457249308377 17 idx: 18
name: i18
dist: 0.22738736048
support: 1
height: 0.655569615686 18 idx: 21
name: i21
dist: 1.07602774143
support: 1
height: 0.882956976166 26 idx: 22
name: i22
dist: 0.532711306491
support: 1
height: 1.4262734111 19 idx: 26
name: i26
dist: 0.869087521378
support: 1
height: 1.95898471759 27 idx: 19
name: i19
dist: 0.274535906905
support: 1
height: 0.432209915217 21 idx: 20
name: i20
dist: 0.0749539538796
support: 1
height: 0.631791868242 22 idx: 23
name: i23
dist: 0.839897470643
support: 1
height: 0.706745822122 20 idx: 24
name: i24
dist: 0.712653023292
support: 1
height: 0.833990269474 23 idx: 27
name: i27
dist: 1.28142894621
support: 1
height: 1.54664329277 30 idx: 30
name: i30
dist: 0.658595163064
support: 1
height: 2.82807223897 29 idx: 25
name: i25
dist: 2.92433383714
support: 1
height: 0.249037539719 24 idx: 28
name: i28
dist: 0.211732229705
support: 1
height: 3.17337137686 25 idx: 29
name: i29
dist: 1.93967988247
support: 1
height: 1.44542372409 28 idx: 31
name: i31
dist: 0.908595163064
support: 1
height: 3.38510360657 31 idx: 32
name: i32
dist: 0.0
support: 100
height: 4.29369876963 32

In [14]:
random.seed(123)
tree = toytree.rtree(17)
tree.tree.convert_to_ultrametric()
samptree, sampdict = sample_branches(tree,returndict="both")
print("Top node height should be: "+str(sampdict[32]))
print("\n" + "Top node height is instead: " + str(samptree.tree.search_nodes(idx = 32)[0].height))

Top node height should be: 6.02032489067

Top node height is instead: 5.17125487688


### So something is off. Good news is that the dictionary itself seems to be correct.

### The other good news is that this still works relatively quickly for big trees, although some of them then take a very long time. Might be good to build in a "bailout" option.

In [11]:
%%timeit
random.seed(123)
tree = toytree.rtree(70)
tree.tree.convert_to_ultrametric()
print(len(sample_branches(tree,returndict="only"))) ## 68 internal nodes!

68
68
68
68
1 loop, best of 3: 137 ms per loop
